In [11]:
import TimeTagger
import numpy as np

## channel/channel example (photon mean per channel)

In [2]:
class PhotonMean(TimeTagger.CustomMeasurement):
    """
    CustomMeasurement that gets channel_1/channel_2.
    """

    def __init__(self, tagger, channels):
        TimeTagger.CustomMeasurement.__init__(self, tagger)
        self.channels = list(channels)
        self.counts = np.zeros((len(channels),), dtype=np.int32)

        # Each used channel must be registered
        for channel in channels:
            self.register_channel(channel)

        self.clear_impl()

        # At the end of a Measurement construction we must indicate that we
        # have finished
        self.finalize_init()

    def __del__(self):
        # The measurement must be stopped before deconstruction to avoid
        # concurrent measure() calls
        self.stop()

    def getData(self):
        # lock this instance to avoid conflicting results while measure is
        # running apart.
        with self.mutex:
            counts = np.array(self.counts)
        return counts[0]/counts[1]

    def getIndex(self):
        # this method does not depend on the internal state, so there is no
        # need for a lock
        return list(self.channels)

    def clear_impl(self):
        # the lock is already acquired
        self.counts *= 0

    def on_start(self):
        # the lock is already acquired
        pass

    def on_stop(self):
        # the lock is already acquired
        pass

    def process(self, incoming_tags, begin_time, end_time):
        # the lock is already acquired
        # self.data is provided as reference, so it must not be accessed
        # anywhere else without locking the mutex.
        # incoming_tags is provided as a read-only reference. The storage will
        # be deallocated after this call, so you must not store a reference to
        # this object. Make a copy instead.
        channel_numbers, counts = np.unique(incoming_tags["channel"], return_counts=True)
        for channel_number, count in zip(channel_numbers, counts):
            self.counts[self.channels.index(channel_number)] += count

In [3]:
photon = 4
frame = 3

In [4]:
dump_file = r"Z:\PriyaM\swabian_data\20230712_tissue\test_tissue_2023-07-12_154752.1.ttbin"

In [5]:
tagger = TimeTagger.createTimeTaggerVirtual()

In [6]:
photon_mean = PhotonMean(tagger, channels=[photon, frame])

In [7]:
tagger.setReplaySpeed(-1)
replay = tagger.replay(dump_file)
tagger.waitForCompletion()

True

In [8]:
photon_mean_data = photon_mean.getData()
print("mean photons per frame:", photon_mean_data)


mean photons per frame: 622230.6666666666


In [9]:
photon_mean.clear_impl()